# This tutorial will show you how to find the suitable habitat range for Bristlecone pine using GeoPySpark

This tutorial will focus on GeoPySpark functionality, but you can find more resources and tutorials about GeoNotebooks [here](https://github.com/OpenGeoscience/geonotebook/tree/master/notebooks).

### Suitability analysis is a classic GIS case study that enables the combination of factors to return a desired result 
This tutorial sets the premise that you are interested in two factors for locating Bristlecone pines:
- Located between 3,000 and 4,000 meters
- Located on a south facing slope
     

In [ ]:
import geopyspark as gps
from pyspark import SparkContext

You will need to set up a spark context. To learn more about what that means take a look [here](https://spark.apache.org/docs/latest/programming-guide.html#initializing-spark)

In [ ]:
conf=gps.geopyspark_conf(appName="BristleConePine")
conf.set('spark.ui.enabled', True)
sc = SparkContext(conf = conf)

Retrieving an elevation .tif from AWS S3:

In [ ]:
elev_rdd = gps.geotiff.get(
    layer_type='spatial', 
    uri='s3://geopyspark-demo/elevation/ca-elevation.tif')

## Tile, reproject, pyramid:

In [ ]:
elev_tiled_rdd = elev_rdd.tile_to_layout(
    layout=gps.GlobalLayout(), 
    target_crs=3857)
elev_pyramided_rdd = elev_tiled_rdd.pyramid().cache()

Imports for creating a TMS server capable of serving layers with custom colormaps

In [ ]:
from geopyspark.geotrellis.color import get_colors_from_matplotlib
elev_histo        = elev_pyramided_rdd.get_histogram()
elev_colors       = get_colors_from_matplotlib('viridis', 100)
elev_color_map    = gps.ColorMap.from_histogram(elev_histo, elev_colors)

In [ ]:
elev_tms = gps.TMS.build(elev_pyramided_rdd, elev_color_map)
elev_tms.bind('0.0.0.0')

Display the tiles in an embedded [Folium](https://python-visualization.github.io/folium/) map:

In [ ]:
import folium

map_center = [37.75, -118.85]
zoom = 7

m = folium.Map(location=map_center, zoom_start=zoom)
folium.TileLayer(tiles="Stamen Terrain", overlay=False).add_to(m)
folium.TileLayer(tiles=elev_tms.url_pattern, attr="GeoPySpark", overlay=True).add_to(m)
m

Classify the elevation such that values of interest (between 3,000 and 4,000 meters) return a value of 1.

In [ ]:
# use: elev_reprojected_rdd
elev_reclass_pre = elev_tiled_rdd.reclassify({1000:2, 2000:2, 3000:2, 4000:1, 5000:2}, int)
elev_reclass_rdd = elev_reclass_pre.reclassify({1:1}, int)
elev_reclass_pyramid_rdd = elev_reclass_rdd.pyramid()

In [ ]:
elev_reclass_histo = elev_reclass_pyramid_rdd.get_histogram()

In [ ]:
#elev_reclass_color_map = ColorMap.from_histogram(sc, elev_reclass_histo, get_breaks(sc, 'Viridis', num_colors=100))
elev_reclass_color_map = gps.ColorMap.from_colors(
    breaks =[1], 
    color_list = [0xff000080])

In [ ]:
elev_reclass_tms = gps.TMS.build(elev_reclass_pyramid_rdd, elev_reclass_color_map)
elev_reclass_tms.bind('0.0.0.0')

In [ ]:
m2 = folium.Map(location=map_center, zoom_start=zoom)
folium.TileLayer(tiles="Stamen Terrain", overlay=False).add_to(m2)
folium.TileLayer(tiles=elev_tms.url_pattern, attr='GeoPySpark', name="Elevation", overlay=True).add_to(m2)
folium.TileLayer(tiles=elev_reclass_tms.url_pattern, attr='GeoPySpark', name="High Elevation Areas", overlay=True).add_to(m2)
folium.LayerControl().add_to(m2)
m2

Focal operation: aspect. To find south facing slopes

In [ ]:
# square_neighborhood = Square(extent=1)
aspect_rdd = elev_tiled_rdd.focal(
    gps.Operation.ASPECT, 
    gps.Neighborhood.SQUARE, 1)

In [ ]:
aspect_pyramid_rdd  = aspect_rdd.pyramid()
aspect_histo        = aspect_pyramid_rdd.get_histogram()
aspect_color_map    = gps.ColorMap.from_histogram(aspect_histo, get_colors_from_matplotlib('viridis', num_colors=256))

In [ ]:
aspect_tms = gps.TMS.build(aspect_pyramid_rdd, aspect_color_map)
aspect_tms.bind('0.0.0.0')

In [ ]:
m3 = folium.Map(tiles='Stamen Terrain', location=map_center, zoom_start=zoom)
folium.TileLayer(tiles=aspect_tms.url_pattern, attr='GeoPySpark', name="High Elevation Areas", overlay=True).add_to(m3)
m3

In [ ]:
aspect_tms.unbind()

Reclassify values such that values between 120 and 240 degrees (south) have a value of 1

In [ ]:
aspect_reclass_pre  = aspect_rdd.reclassify({120:2, 240:1, 360: 2}, int)
aspect_reclass      = aspect_reclass_pre.reclassify({1:1}, int)

In [ ]:
aspect_reclass_pyramid_rdd = aspect_reclass.pyramid()
aspect_reclass_histo       = aspect_reclass_pyramid_rdd.get_histogram()
aspect_reclass_color_map   = gps.ColorMap.from_histogram(aspect_reclass_histo, get_colors_from_matplotlib('viridis', num_colors=256))

In [ ]:
aspect_reclass_tms = gps.TMS.build(aspect_reclass_pyramid_rdd, aspect_reclass_color_map)
aspect_reclass_tms.bind('0.0.0.0')

In [ ]:
m4 = folium.Map(tiles='Stamen Terrain', location=map_center, zoom_start=zoom)
folium.TileLayer(tiles=aspect_reclass_tms.url_pattern, attr='GeoPySpark', name="High Elevation Areas", overlay=True).add_to(m4)
m4

In [ ]:
aspect_reclass_tms.unbind()

Now add the values togehter to find the suitable range:

In [ ]:
added = elev_reclass_pyramid_rdd + aspect_reclass_pyramid_rdd
added_histo = added.get_histogram()
added_color_map = gps.ColorMap.from_histogram(added_histo, get_colors_from_matplotlib('viridis', num_colors=256))

In [ ]:
added_tms = gps.TMS.build(added, added_color_map)
added_tms.bind('0.0.0.0')

In [ ]:
m5 = folium.Map(tiles='Stamen Terrain', location=map_center, zoom_start=zoom)
folium.TileLayer(tiles=added_tms.url_pattern, attr='GeoPySpark', name="High Elevation Areas", overlay=True).add_to(m5)
m5

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
v = elev_tiled_rdd.lookup(342,787)
plt.imshow(v[0].cells[0])